In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

g:\AgenticAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings.embed_query("hello AI")

[-0.033388201147317886,
 0.034539803862571716,
 0.059474579989910126,
 0.05928615480661392,
 -0.06353538483381271,
 -0.06819586455821991,
 0.08823323249816895,
 0.0344407856464386,
 -0.03278515860438347,
 -0.015814995393157005,
 0.020981667563319206,
 -0.018340280279517174,
 -0.03983214125037193,
 -0.08047076314687729,
 -0.014469259418547153,
 0.03326486423611641,
 0.01425926387310028,
 -0.03404999524354935,
 -0.142915740609169,
 -0.023083265870809555,
 -0.02138010412454605,
 0.0026335634756833315,
 -0.04729275032877922,
 -0.010752726346254349,
 -0.06866799294948578,
 0.03112499974668026,
 0.07594592869281769,
 0.0011283049825578928,
 0.011631982401013374,
 -0.036039214581251144,
 0.04483765363693237,
 0.01839079149067402,
 0.12672799825668335,
 -0.001359735382720828,
 0.008206715807318687,
 0.06909963488578796,
 -0.0807635709643364,
 -0.05841311439871788,
 0.05375451594591141,
 0.026227623224258423,
 -0.00682859867811203,
 -0.05635840445756912,
 0.0032929950393736362,
 -0.072501808404

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [8]:
my_query="Narendra modi is prime minister of india?"

In [9]:
document_embedding=embeddings.embed_documents(documents)

In [14]:
len(document_embedding)

3

In [15]:
document_embedding

[[0.11998696625232697,
  -0.021302612498402596,
  -0.04288085177540779,
  0.06645581871271133,
  -0.06435241550207138,
  -0.04424869641661644,
  0.02240842767059803,
  -0.04987306892871857,
  -0.023437613621354103,
  -0.03397210314869881,
  -0.014048037119209766,
  -0.06065928936004639,
  -0.0039068227633833885,
  -0.017782099545001984,
  -0.04797104746103287,
  -0.06668158620595932,
  0.004103198181837797,
  -0.013092797249555588,
  0.04439776763319969,
  0.02235068380832672,
  0.009459559805691242,
  -0.020564520731568336,
  -0.0003356117522343993,
  -0.005685788579285145,
  0.05558694899082184,
  0.02512318082153797,
  -0.0028171385638415813,
  0.008758987300097942,
  0.003255277406424284,
  -0.015963422134518623,
  0.014263655990362167,
  -0.11220847070217133,
  0.08968555927276611,
  -0.03108377195894718,
  -0.024223867803812027,
  0.006152097601443529,
  0.08058716356754303,
  0.018250001594424248,
  0.05568313971161842,
  0.016702702268958092,
  0.01589600369334221,
  0.00034111

In [15]:
query_embedding=embeddings.embed_query(my_query)

In [16]:
len(query_embedding)

384

In [17]:
cosine_similarity([query_embedding],document_embedding)

array([[0.11756665, 0.3432456 , 0.8141324 ]])

In [18]:
from sklearn.metrics.pairwise import euclidean_distances

In [19]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32848288, 1.14608416, 0.60970094]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [21]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [22]:
index=faiss.IndexFlatL2(384)
 

In [23]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001BF8E5CB420> >

In [24]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [25]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['90ef2a26-a4a4-4538-b3f1-ab791fb88f45',
 '67b530ca-a880-4f22-87f7-ab35055b53ea',
 'ef82a927-f962-4431-9e45-ad9bb9970fa0']

In [26]:
vector_store.index_to_docstore_id

{0: '90ef2a26-a4a4-4538-b3f1-ab791fb88f45',
 1: '67b530ca-a880-4f22-87f7-ab35055b53ea',
 2: 'ef82a927-f962-4431-9e45-ad9bb9970fa0'}

In [29]:
results = vector_store.similarity_search("Tell me about AI", k=2)

In [30]:
results


[Document(id='67b530ca-a880-4f22-87f7-ab35055b53ea', metadata={}, page_content='AI is powerful'),
 Document(id='90ef2a26-a4a4-4538-b3f1-ab791fb88f45', metadata={}, page_content='AI is future')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [31]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [32]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [33]:
vector_store.add_documents(documents=documents)

['aeb1be7b-25db-48d4-8c1e-9300cb721ba1',
 'bf03b77f-a916-4b9d-a118-82d4a577f70c',
 'ed4ed77b-3f1a-428f-83fe-0767b541a823',
 '91104530-c1f1-4d36-9531-268d7f63ffe5',
 '15ff63b8-c567-448d-84a1-000bf29aefa6',
 '80b0ab9e-f4c9-4128-8dff-7d70c4b909b8',
 '4d5e2940-53af-40aa-824f-7d53d897488b',
 '41478929-6e59-4bcd-b8d6-37a400856a72',
 'e6d7adc4-fa9e-4792-9d0f-1c56e6177b59',
 '59ecbdba-a5e1-4d85-ba5c-1651a164a269']

In [34]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(id='ed4ed77b-3f1a-428f-83fe-0767b541a823', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='41478929-6e59-4bcd-b8d6-37a400856a72', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [35]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
    
)

[Document(id='ed4ed77b-3f1a-428f-83fe-0767b541a823', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='41478929-6e59-4bcd-b8d6-37a400856a72', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='59ecbdba-a5e1-4d85-ba5c-1651a164a269', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='aeb1be7b-25db-48d4-8c1e-9300cb721ba1', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [36]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [37]:
result[0].metadata

{'source': 'news'}

In [38]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [39]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [40]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='ed4ed77b-3f1a-428f-83fe-0767b541a823', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='41478929-6e59-4bcd-b8d6-37a400856a72', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='59ecbdba-a5e1-4d85-ba5c-1651a164a269', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

In [42]:
# inmemory(server)
# ondisk(server)
# cloud(yet to discuss)

In [50]:
vector_store.save_local("today's class faiss index")

In [43]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [44]:
new_vector_store.similarity_search("langchain")

[Document(id='e6ade172-ce20-4179-861e-147cf6e1c3d5', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='a6e1aa11-4850-438c-acef-78be27abb03a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='b4c2983a-bcf2-4ba0-9182-4e68b530a9aa', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='f3fbd2f5-dd62-4bfc-b03f-02863a8bcc73', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [45]:
from langchain_community.document_loaders import PyPDFLoader

In [47]:
FILE_PATH=r"llama.pdf"

In [48]:
loader=PyPDFLoader(FILE_PATH)

In [49]:
len(loader.load())

77

In [50]:
pages=loader.load()

In [51]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [52]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [53]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [54]:
split_docs = splitter.split_documents(pages)

In [55]:
len(split_docs)

615

In [56]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [57]:
vector_store.add_documents(documents=split_docs)

['31309ab8-f64d-4506-a649-f701e4b024b5',
 '7a34f045-aa23-493c-b62d-d9c29d45f83c',
 '47447ea7-30ec-4102-900f-e63696392a8d',
 '329c689d-e9c2-43ed-93fa-ac46fd1aec84',
 '349020a6-ec03-4e1d-8a96-5fef5d631596',
 '957165d9-3006-4349-8cc5-a31b50a0c20d',
 'cfd4a861-fa22-4f36-ae22-003f8e3f9c2a',
 'b433e156-43b0-431f-9da6-ab2b9ebb618e',
 'b496eaa3-de7f-4cef-b826-bffe97da1b66',
 'a51a17d2-b8f0-420e-82d8-d3e74341025f',
 '32e54329-8cef-43d6-9598-b29ae5a29ea6',
 '51d98186-5000-4c6c-a7e8-bb2fb441a6a3',
 'dffa4898-fb76-40ba-a2ba-3b1e1c8d44b1',
 '749e7918-9ffa-40ac-8a8f-89829d926754',
 'f77c882d-8d4c-404d-a879-d0d907cd41ad',
 '171f2db7-2406-4e23-bb2c-518ab7700daa',
 '892aab31-7c99-44da-b43f-97f954cd7018',
 '6f7ab995-afc5-4d3f-bc16-c592e14a6c28',
 '4e5dea5c-2e5d-4dc9-a36b-501dbb35f856',
 '13d88a87-40bd-4a46-888f-7622482fce53',
 '856db297-b45c-40ae-8b04-529f9d265515',
 '51f78d44-bd56-43db-989a-90c6f37ab10f',
 '68211a5a-fd41-4968-a213-2e56abc6caa7',
 '1a3036ac-0e49-4cb5-8d48-4c6c2b8f9d63',
 'e778fc4e-a3cf-

In [58]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [59]:
retriever.invoke("what is llama model?")

[Document(id='e778fc4e-a3cf-447f-a84b-7a792c28ef59', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'llama.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='be41c3ab-3e95-4c20-98d4-2200a8d8b022', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36

In [80]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [81]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Users\sunny\agenticai-2.0\env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [82]:
import pprint


In [83]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [84]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
context(retriever),prompt(hub),model(google),parser(langchain)

In [86]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [87]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [88]:
rag_chain.invoke("what is llama model?")

'Llama is a large language model developed by Meta.  It has various versions, including Llama 1 and Llama 2, with different parameter sizes.  Llama 2 is designed for commercial and research use and is available under a custom commercial license.'